In [1]:
import re
import os
from time import gmtime, strftime
from datetime import datetime, timedelta
import unicodedata

import pandas as pd
import numpy as np
import requests
import bs4 as bs
from lxml import html
from tqdm import tqdm

In [12]:
ticker_cik_df = pd.DataFrame.from_dict(cik_dict, orient='index')

In [14]:
ticker_cik_df.reset_index(inplace=True)
ticker_cik_df.columns = ['ticker', 'cik']

In [15]:
ticker_cik_df

,ticker,cik
0,aapl,0000320193
1,goog,0001652044


In [19]:
pathname_10k = os.path.join(os.getcwd(), 'file_10k')

In [21]:
browse_url_base_10k = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=%s&type=10-K'
filing_url_base_10k = 'http://www.sec.gov/Archives/edgar/data/%s/%s-index.html'
doc_url_base_10k = 'http://www.sec.gov/Archives/edgar/data/%s/%s/%s'

In [29]:
cik = ticker_cik_df['cik'][0]

In [26]:
res = requests.get(browse_url_base_10k % cik)

In [32]:
soup = bs.BeautifulSoup(res.text, 'lxml')

In [33]:
html_tables = soup.find_all('table')

In [34]:
if len(html_tables)<3:
    os.chdir('..')

In [44]:
filings_table = pd.read_html(str(html_tables[2]), header=0)[0]

In [45]:
filings_table

,Filings,Format,Description,Filing Date,File/Film Number
0,10-K/A,Documents,"[Amend] Annual report [Section 13 and 15(d), n...",2019-02-06,001-3758019569784
1,10-K,Documents Interactive Data,"Annual report [Section 13 and 15(d), not S-K I...",2019-02-05,001-3758019566006
2,10-K,Documents Interactive Data,"Annual report [Section 13 and 15(d), not S-K I...",2018-02-06,001-3758018575994
3,10-K,Documents Interactive Data,"Annual report [Section 13 and 15(d), not S-K I...",2017-02-03,001-3758017569702
4,10-K/A,Documents,"[Amend] Annual report [Section 13 and 15(d), n...",2016-03-29,001-37580161533561
5,10-K,Documents Interactive Data,"Annual report [Section 13 and 15(d), not S-K I...",2016-02-11,001-37580161412149


In [48]:
filings_table['Filings'] = [str(x) for x in filings_table['Filings']]

In [49]:
filings_table = filings_table[(filings_table['Filings'] == '10-K') | (filings_table['Filings'] == '10-K405')]

In [66]:
filings_table['Acc_No'] = [x.replace('\xa0', ' ').split('Acc-no: ')[1].split(' ')[0]
                           for x  in filings_table['Description']]

C:\Users\vhuang\Anaconda3\envs\py36tf2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [97]:
filings_table

,Filings,Format,Description,Filing Date,File/Film Number,Acc_No
1,10-K,Documents Interactive Data,"Annual report [Section 13 and 15(d), not S-K I...",2019-02-05,001-3758019566006,0001652044-19-000004
2,10-K,Documents Interactive Data,"Annual report [Section 13 and 15(d), not S-K I...",2018-02-06,001-3758018575994,0001652044-18-000007
3,10-K,Documents Interactive Data,"Annual report [Section 13 and 15(d), not S-K I...",2017-02-03,001-3758017569702,0001652044-17-000008
5,10-K,Documents Interactive Data,"Annual report [Section 13 and 15(d), not S-K I...",2016-02-11,001-37580161412149,0001652044-16-000012


In [58]:
for index, row in filings_table.iterrows():
    acc_no = str(row['Acc_No'])
    docs_page = requests.get(filing_url_base_10k % (cik, acc_no))
    
    if docs_page.status_code != 200:
        os.chdir(cik)
        continue
    
    docs_page

'Annual report [Section 13 and 15(d), not S-K Item 405]Acc-no: 0001652044-19-000004\xa0(34 Act) Size: 14 MB'

In [65]:
x.replace('\xa0', ' ').split('Acc-no: ')[1].split(' ')[0]

'0001652044-19-000004'

In [107]:
acc_no = str(filings_table['Acc_No'][1])

In [110]:
docs_page = requests.get(filing_url_base_10k % (cik, acc_no))

In [112]:
docs_page_soup = bs.BeautifulSoup(docs_page.text, 'lxml')

In [113]:
docs_html_tables = docs_page_soup.find_all('table')

In [116]:
if len(docs_html_tables)==0:
    pass
docs_table = pd.read_html(str(docs_html_tables[0]), header=0)[0]

In [117]:
docs_table

,Seq,Description,Document,Type,Size
0,1.0,10-K,goog10-kq42018.htm,10-K,2738368
1,2.0,EXHIBIT 21.01,googexhibit2101q42018.htm,EX-21.01,4474
2,3.0,EXHIBIT 23.01,googexhibit2301q42018.htm,EX-23.01,8574
3,4.0,EXHIBIT 31.01,googexhibit3101q42018.htm,EX-31.01,12394
4,5.0,EXHIBIT 31.02,googexhibit3102q42018.htm,EX-31.02,12643
5,6.0,EXHIBIT 32.01,googexhibit3201q42018.htm,EX-32.01,9821
6,13.0,NaN,chart-7c2ecf1250415c72a9ca01.jpg,GRAPHIC,79730
7,14.0,NaN,chart-d02c77e5831a57f6997a01.jpg,GRAPHIC,77686
8,NaN,Complete submission text file,0001652044-19-000004.txt,NaN,14192672


In [118]:
cik

'0000320193'

In [119]:
os.getcwd()

'C:\\Users\\vhuang\\Documents\\GitHub\\WordEmbedding-Word2Vec-GloVe'